In [247]:
import asyncio
from playwright.async_api import async_playwright
import time
import random

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?i=computers&bbn=429886031&rh=n%3A429886031%2Cp_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&dc&page=2&qid=1698924010&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    await asyncio.sleep(random.uniform(1, 5)) 
hrefs


['/2022-Apple-MacBook-laptop-chip/dp/B0B3BM4DTG/ref=sr_1_39?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-39',
 '/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2WLJ2P/ref=sr_1_48?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-48',
 '/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_36?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-36',
 '/Apple-MacBook-13-inch-Storage-Renewed/dp/B0BJ41FV7N/ref=sr_1_37?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-37',
 '/Apple-MacBook-13in-MD101-2012/dp/B07B3WTJ3J/ref=sr_1_41?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-41',
 '/Apple-MacBook-dual-core-1-6GHz-

In [248]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await asyncio.sleep(random.uniform(1, 5))

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'Macbook'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BM4DTG/ref=sr_1_39?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-39
2 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2WLJ2P/ref=sr_1_48?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-48
3 / 240
/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_36?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-36
4 / 240
/Apple-MacBook-13-inch-Storage-Renewed/dp/B0BJ41FV7N/ref=sr_1_37?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-37
5 / 240
/Apple-MacBook-13in-MD101-2012/dp/B07B3WTJ3J/ref=sr_1_41?qid=1699086760&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-41
6 / 240
/Apple

43 / 240
/Apple-MacBook-512GB-Space-Renewed/dp/B0BJ35C47J/ref=sr_1_34?qid=1699086765&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-34
44 / 240
/Apple-Macbook-13-inch-Storage-Starlight/dp/B0CJMQ6F1L/ref=sr_1_35?qid=1699086765&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35
45 / 240
/2019-Apple-MacBook-Pro-16/dp/B0BB3BK6SG/ref=sr_1_44?qid=1699086765&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-44
46 / 240
/Apple-MacBook-laptop-10%E2%80%91core-16%E2%80%91core/dp/B0BSHVC2QV/ref=sr_1_38?qid=1699086765&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-38
47 / 240
/Apple-MacBook-13-inch-Storage-Renewed/dp/B0BJ41FV7N/ref=sr_1_37?qid=1699086765&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=co

84 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BM4DTG/ref=sr_1_39?qid=1699086775&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-39
85 / 240
/Apple-MacBook-Air-Late-2018/dp/B07RPC252K/ref=sr_1_46?qid=1699086775&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-46
86 / 240
/Apple-MacBook-Pro-13-2017/dp/B08SC2V9TX/ref=sr_1_31?qid=1699086775&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-31
87 / 240
/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_36?qid=1699086775&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-36
88 / 240
/Apple-MacBook-laptop-10%E2%80%91core-16%E2%80%91core/dp/B0BSHVC2QV/ref=sr_1_38?qid=1699086775&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-38
89 

126 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0BGG744XY/ref=sr_1_25?qid=1699086786&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-25
127 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BJD4F4/ref=sr_1_29?qid=1699086786&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-29
128 / 240
/Apple-Inc-MacBook-Pro-Touch/dp/B085TMSYWW/ref=sr_1_45?qid=1699086786&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-45
129 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2SKG4B/ref=sr_1_40?qid=1699086786&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-40
130 / 240
/Apple-MacBook-13-inch-Storage-Renewed/dp/B0BJ41FV7N/ref=sr_1_37?qid=1699086786&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-37
131 / 240

167 / 240
/Apple-MacBook-Pro-13-2017/dp/B08SC2V9TX/ref=sr_1_31?qid=1699086789&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-31
168 / 240
/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_36?qid=1699086789&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-36
169 / 240
/Apple-MacBook-512GB-Space-Renewed/dp/B0BJ35C47J/ref=sr_1_34?qid=1699086796&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-34
170 / 240
/Apple-MacBook-13in-MD101-2012/dp/B07B3WTJ3J/ref=sr_1_41?qid=1699086796&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-41
171 / 240
/Apple-MacBook-MD101-A1278-2012/dp/B07B3M8KT9/ref=sr_1_28?qid=1699086796&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-28
172 / 240
/Appl

209 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BK2K8Y/ref=sr_1_33?qid=1699086802&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-33
210 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2WTNQ6/ref=sr_1_42?qid=1699086802&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-42
211 / 240
/Apple-Macbook-13-inch-Storage-Starlight/dp/B0CJMQ6F1L/ref=sr_1_35?qid=1699086802&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35
212 / 240
/Apple-Inc-MacBook-Pro-Touch/dp/B085TMSYWW/ref=sr_1_45?qid=1699086802&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-45
213 / 240
/Apple-MacBook-512GB-Space-Renewed/dp/B0BJ35C47J/ref=sr_1_34?qid=1699086802&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-34
214 

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,Early 2019 Apple MacBook Pro with 1.4GHz Intel...,589..,4.1 out of 5,48 global ratings,https://www.amazon.co.uk/Apple-Inc-MacBook-Pro...,Value not found,Apple,MacBook Pro,Mac OS 9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1465,Early 2019 Apple MacBook Pro with 1.4GHz Intel...,589..,4.1 out of 5,48 global ratings,https://www.amazon.co.uk/Apple-Inc-MacBook-Pro...,Value not found,Apple,MacBook Pro,Mac OS 9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1466,Early 2019 Apple MacBook Pro with 1.4GHz Intel...,589..,4.1 out of 5,48 global ratings,https://www.amazon.co.uk/Apple-Inc-MacBook-Pro...,Value not found,Apple,MacBook Pro,Mac OS 9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1467,Early 2019 Apple MacBook Pro with 1.4GHz Intel...,589..,4.1 out of 5,48 global ratings,https://www.amazon.co.uk/Apple-Inc-MacBook-Pro...,Value not found,Apple,MacBook Pro,Mac OS 9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [249]:
df.Search.value_counts()

iPhone         648
AirPods        325
iPad           261
Macbook        144
Apple Watch     81
Name: Search, dtype: int64

In [251]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=iphone&i=electronics&rh=n%3A560820%2Cp_72%3A419156031&dc&page=2&qid=1698924321&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    await asyncio.sleep(random.uniform(1, 5)) 
hrefs


['/Apple-iPhone-128GB-Purple-Renewed/dp/B09CLGDGTL/ref=sr_1_48?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-48',
 '/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD6G/ref=sr_1_42?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-42',
 '/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_45?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-45',
 '/Apple-iPhone-Pro-128GB-Graphite/dp/B08PCDHMWQ/ref=sr_1_35?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-35',
 '/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_25?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25',
 '/Apple-iPhone-64GB-Red-Renewed/dp/B07P5NGFR7/ref=sr_1_38?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-38',
 '/Apple-iPhone-mini-64GB-Black/dp/

In [252]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'iPhone'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-iPhone-128GB-Purple-Renewed/dp/B09CLGDGTL/ref=sr_1_48?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-48
2 / 240
/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD6G/ref=sr_1_42?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-42
3 / 240
/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_45?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-45
4 / 240
/Apple-iPhone-Pro-128GB-Graphite/dp/B08PCDHMWQ/ref=sr_1_35?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-35
5 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_25?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25
6 / 240
/Apple-iPhone-64GB-Red-Renewed/dp/B07P5NGFR7/ref=sr_1_38?keywords=iphone&qid=1699095488&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-38
7 / 240
/Ap

52 / 240
/Apple-iPhone-mini-128GB-Blue/dp/B08PCDNC9W/ref=sr_1_47?keywords=iphone&qid=1699095497&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-47
53 / 240
/Apple-iPhone-12-64GB-Renewed-Purple/dp/B0B42V45SV/ref=sr_1_46?keywords=iphone&qid=1699095497&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-46
54 / 240
/Xiaomi-display-ultra-thin-Version-warranty-Black/dp/B0BYVKMXJW/ref=sr_1_38?keywords=iphone&qid=1699095497&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-38
55 / 240
/Apple-iPhone-14-Pro-128/dp/B0BDJ61GFY/ref=sr_1_31?keywords=iphone&qid=1699095497&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-31
56 / 240
/Apple-iPhone-128GB-Purple-Renewed/dp/B09CLGDGTL/ref=sr_1_48?keywords=iphone&qid=1699095497&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-48
57 / 240
/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD6G/ref=sr_1_42?keywords=iphone&qid=1699095497&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-42
58 / 240
/Ap

103 / 240
/Apple-iPhone-12-64GB-Green/dp/B08PCCS7PH/ref=sr_1_29?keywords=iphone&qid=1699095510&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-29
104 / 240
/Apple-iPhone-15-128-GB/dp/B0CHX3RPHZ/ref=sr_1_33?keywords=iphone&qid=1699095510&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33
105 / 240
/Apple-iPhone-mini-64GB-Blue/dp/B08PCD1417/ref=sr_1_27?keywords=iphone&qid=1699095510&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-27
106 / 240
/Apple-iPhone-11-64GB-White/dp/B08L6XKFRY/ref=sr_1_30?keywords=iphone&qid=1699095510&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-30
107 / 240
/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_45?keywords=iphone&qid=1699095510&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-45
108 / 240
/Apple-iPhone-Pro-128GB-Graphite/dp/B08PCDHMWQ/ref=sr_1_36?keywords=iphone&qid=1699095510&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-36
109 / 240
/Xiao

154 / 240
/Apple-iPhone-mini-128GB-Blue/dp/B08PCDNC9W/ref=sr_1_47?keywords=iphone&qid=1699095519&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-47
155 / 240
/Apple-iPhone-mini-64GB-Blue/dp/B08PCD1417/ref=sr_1_27?keywords=iphone&qid=1699095519&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-27
156 / 240
/Apple-iPhone-14-Pro-128/dp/B0BDJ61GFY/ref=sr_1_31?keywords=iphone&qid=1699095519&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-31
157 / 240
/Apple-iPhone-Pro-Max-256/dp/B0BDJGV887/ref=sr_1_44?keywords=iphone&qid=1699095519&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-44
158 / 240
/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_45?keywords=iphone&qid=1699095519&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-45
159 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_43?keywords=iphone&qid=1699095519&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-43
160 / 240
/Apple-iPho

205 / 240
/Apple-iPhone-Pro-128GB-Silver/dp/B08PCCM1GC/ref=sr_1_26?keywords=iphone&qid=1699095529&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-26
206 / 240
/Apple-iPhone-15-Pro-256/dp/B0CHX5R7K3/ref=sr_1_32?keywords=iphone&qid=1699095529&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-32
207 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_25?keywords=iphone&qid=1699095529&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25
208 / 240
/Apple-iPhone-15-128-GB/dp/B0CHX3RPHZ/ref=sr_1_30?keywords=iphone&qid=1699095529&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-30
209 / 240
/Apple-iPhone-mini-64GB-Black/dp/B08PCF2P4R/ref=sr_1_34?keywords=iphone&qid=1699095529&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-34
210 / 240
/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_35?keywords=iphone&qid=1699095529&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-35
211 / 240
/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,Apple iPhone 15 (128 GB) - Blue,799..,4.4 out of 5,219 global ratings,https://www.amazon.co.uk/Apple-iPhone-15-128-G...,Value not found,Apple,IPhone 15,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1476,Apple iPhone 15 (128 GB) - Blue,799..,4.4 out of 5,219 global ratings,https://www.amazon.co.uk/Apple-iPhone-15-128-G...,Value not found,Apple,IPhone 15,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,"Apple iPhone SE 2nd Generation, 64GB, Black (R...",110..,4.3 out of 5,726 global ratings,https://www.amazon.co.uk/Apple-iPhone-Generati...,Value not found,Apple,IPhone SE (2nd Gen),IOS 12,4G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,"Apple iPhone SE 2nd Generation, 64GB, Black (R...",110..,4.3 out of 5,726 global ratings,https://www.amazon.co.uk/Apple-iPhone-Generati...,Value not found,Apple,IPhone SE (2nd Gen),IOS 12,4G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
df.Search.value_counts()

iPhone         659
AirPods        325
iPad           261
Macbook        144
Apple Watch     81
Name: Search, dtype: int64

In [254]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=ipad&i=electronics&page=2&crid=2SLN76UU27TLB&qid=1698924622&sprefix=ipad%2Celectronics%2C98&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    await asyncio.sleep(random.uniform(1, 5)) 
hrefs


['/fire-hd-8-kids-pro/dp/B09BG69PNM/ref=sr_1_42?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-42',
 '/Apple-iPad-10-2-128GB-Wi-Fi/dp/B08N8L8W21/ref=sr_1_46?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-46',
 '/2022-Apple-10-9-inch-iPad-Air-Wi-Fi/dp/B09V4K8868/ref=sr_1_34?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-34',
 '/Apple-iPad-10-2-32GB-4G/dp/B0845WKG2P/ref=sr_1_40?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-40',
 '/Samsung-Android-Version-Manufacturer-Warranty/dp/B0BD4XHGQN/ref=sr_1_48?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-48',
 '/Apple-iPad-12-9-inch-Wi-Fi-256GB/dp/B086M8CGRT/ref=sr_1_45?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-45',
 '/2021-Apple-10

In [255]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'iPad'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/fire-hd-8-kids-pro/dp/B09BG69PNM/ref=sr_1_42?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-42
2 / 240
/Apple-iPad-10-2-128GB-Wi-Fi/dp/B08N8L8W21/ref=sr_1_46?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-46
3 / 240
/2022-Apple-10-9-inch-iPad-Air-Wi-Fi/dp/B09V4K8868/ref=sr_1_34?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-34
4 / 240
/Apple-iPad-10-2-32GB-4G/dp/B0845WKG2P/ref=sr_1_40?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-40
5 / 240
/Samsung-Android-Version-Manufacturer-Warranty/dp/B0BD4XHGQN/ref=sr_1_48?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-48
6 / 240
/Apple-iPad-12-9-inch-Wi-Fi-256GB/dp/B086M8CGRT/ref=sr_1_45?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096597&s=electronics&sprefix=ipad%2Celectronics%2C98&

50 / 240
/Apple-Generation-128GB-Retail-Renewed/dp/B084H2HDND/ref=sr_1_30?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096609&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-30
51 / 240
/Apple-iPad-10-2-inch-Wi-Fi-Cellular/dp/B08N8D3XHB/ref=sr_1_34?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096609&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-34
52 / 240
/Apple-2022-10-9-inch-iPad-Wi-Fi/dp/B0BJMDHJWG/ref=sr_1_41?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096609&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-41
53 / 240
/BYYBUO-SmartPad-A10-Quad-Core-Touchscreen/dp/B09TQYQL1Z/ref=sr_1_38?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096609&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-38
54 / 240
/Apple-iPad-Mini-Space-Refurbished-Grey/dp/B01LZ2GACT/ref=sr_1_36?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096609&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-36
55 / 240
/fire-hd-8-kids-pro/dp/B09BG69PNM/ref=sr_1_42?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096609&s=electronics&sprefi

99 / 240
/Apple-iPad-Air-16GB-Silver/dp/B077J7H62P/ref=sr_1_27?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096620&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-27
100 / 240
/Apple-11-inch-iPad-Pro-Wi-Fi-128GB/dp/B0BJLDMTM3/ref=sr_1_47?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096620&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-47
101 / 240
/Apple-iPad-Air-32GB-Wi-Fi/dp/B07GT4NV6V/ref=sr_1_37?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096620&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-37
102 / 240
/Apple-2022-10-9-inch-iPad-Wi-Fi/dp/B0BJMDHJWG/ref=sr_1_41?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096620&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-41
103 / 240
/2021-Apple-10-2-inch-Wi-Fi-256GB/dp/B09N6LHQSB/ref=sr_1_45?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096620&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-45
104 / 240
/2021-Apple-10-2-inch-Wi-Fi-Cellular/dp/B09G9DLBR8/ref=sr_1_33?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096620&s=electronics&sprefix=ipad%2Cele

147 / 240
/Apple-iPad-Air-16GB-Wi-Fi/dp/B00LJPHPGW/ref=sr_1_29?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096632&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-29
148 / 240
/Apple-2022-10-9-inch-iPad-Wi-Fi/dp/B0BJMDHJWG/ref=sr_1_41?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096632&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-41
149 / 240
/Apple-iPad-10-2-32GB-4G/dp/B0845WKG2P/ref=sr_1_40?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096632&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-40
150 / 240
/2021-Apple-10-2-inch-Wi-Fi-Cellular/dp/B09G9DLBR8/ref=sr_1_33?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096632&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-33
151 / 240
/Apple-iPad-Air-32GB-Wi-Fi/dp/B07GT4NV6V/ref=sr_1_37?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096632&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-37
152 / 240
/Apple-iPad-Air-16GB-Silver/dp/B077J7H62P/ref=sr_1_27?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096632&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=

195 / 240
/Apple-iPad-10-2-inch-Wi-Fi-Cellular/dp/B085WZYBGG/ref=sr_1_28?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096644&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-28
196 / 240
/Apple-iPad-16GB-Wi-Fi-Refurbished/dp/B074JFBNC6/ref=sr_1_27?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096644&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-27
197 / 240
/Apple-iPad-10-2-inch-Wi-Fi-Cellular/dp/B08N8D3XHB/ref=sr_1_35?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096644&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-35
198 / 240
/Apple-11-inch-iPad-Pro-Wi-Fi-128GB/dp/B0BJLDMTM3/ref=sr_1_47?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096644&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-47
199 / 240
/iPad-Air-16GB-WiFi-Cellular/dp/B09LD1D91V/ref=sr_1_32?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096644&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-32
200 / 240
/Apple-iPad-10-2-128GB-Wi-Fi/dp/B08N8L8W21/ref=sr_1_46?crid=2SLN76UU27TLB&keywords=ipad&qid=1699096644&s=electronics&sprefix=i

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,Apple iPhone 15 (128 GB) - Blue,799..,4.4 out of 5,219 global ratings,https://www.amazon.co.uk/Apple-iPhone-15-128-G...,Value not found,Apple,IPhone 15,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1476,Apple iPhone 15 (128 GB) - Blue,799..,4.4 out of 5,219 global ratings,https://www.amazon.co.uk/Apple-iPhone-15-128-G...,Value not found,Apple,IPhone 15,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,"Apple iPhone SE 2nd Generation, 64GB, Black (R...",110..,4.3 out of 5,726 global ratings,https://www.amazon.co.uk/Apple-iPhone-Generati...,Value not found,Apple,IPhone SE (2nd Gen),IOS 12,4G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,"Apple iPhone SE 2nd Generation, 64GB, Black (R...",110..,4.3 out of 5,726 global ratings,https://www.amazon.co.uk/Apple-iPhone-Generati...,Value not found,Apple,IPhone SE (2nd Gen),IOS 12,4G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
df.Search.value_counts()

iPhone         659
AirPods        325
iPad           261
Macbook        144
Apple Watch     81
Name: Search, dtype: int64

In [257]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=AirPods&i=electronics&rh=n%3A4085731%2Cp_72%3A419156031&dc&page=2&qid=1698924837&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    await asyncio.sleep(random.uniform(1, 5))
hrefs


['/Sony-WF-C500-Wireless-Headphones-Built-Black/dp/B09FKGJ1CB/ref=sr_1_38?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38',
 '/Headphones-Bluetooth-Earphones-AI-Enhanced-Cancelling-Black/dp/B0BVQG2LVW/ref=sr_1_40?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40',
 '/JLab-Wireless-Bluetooth-Earphones-Microphone-Teal/dp/B09HN4DHFL/ref=sr_1_39?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-39',
 '/Wireless-Earbuds-Bluetooth-Headphones-Waterproof-White/dp/B08J7B4GMC/ref=sr_1_43?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43',
 '/Wireless-Bluetooth-Headphones-Waterproof-Earphones-UK-3L/dp/B0CHRT2NYM/ref=sr_1_32?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32',
 '/Headphones-Bluetooth-Cancellation-Waterproof-Earphones-Black/dp/B0BT87ZYJG/ref=sr_1_4

In [258]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'AirPods'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 244
/Sony-WF-C500-Wireless-Headphones-Built-Black/dp/B09FKGJ1CB/ref=sr_1_38?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38
2 / 244
/Headphones-Bluetooth-Earphones-AI-Enhanced-Cancelling-Black/dp/B0BVQG2LVW/ref=sr_1_40?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40
3 / 244
/JLab-Wireless-Bluetooth-Earphones-Microphone-Teal/dp/B09HN4DHFL/ref=sr_1_39?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-39
4 / 244
/Wireless-Earbuds-Bluetooth-Headphones-Waterproof-White/dp/B08J7B4GMC/ref=sr_1_43?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43
5 / 244
/Wireless-Bluetooth-Headphones-Waterproof-Earphones-UK-3L/dp/B0CHRT2NYM/ref=sr_1_32?keywords=AirPods&qid=1699096849&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32
6 / 244
/Headphones-Bluetooth-Cancellation-Waterproof-Earphones-Blac

43 / 244
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-White/dp/B0BG579GDX/ref=sr_1_36?keywords=AirPods&qid=1699096855&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-36
44 / 244
/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Rose-Gold/dp/B0C5DKXSVD/ref=sr_1_46?keywords=AirPods&qid=1699096855&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-46
45 / 244
/soundcore-A3919-parent-Black/dp/B07SJR6HL3/ref=sr_1_50?keywords=AirPods&qid=1699096855&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-50
46 / 244
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B0B824NB2D/ref=sr_1_35?keywords=AirPods&qid=1699096855&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
47 / 244
/Certified-Light-Weight-Headphones-Microphone-Cancelling/dp/B0BPX31YB3/ref=sr_1_38?keywords=AirPods&qid=1699096855&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38
48 / 244
/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Black

86 / 244
/Soundcore-Cancelling-AI-Enhanced-Transparency-Bluetooth-Black/dp/B09PBPDQXG/ref=sr_1_30?keywords=AirPods&qid=1699096868&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-30
87 / 244
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B0C1YYSXH8/ref=sr_1_47?keywords=AirPods&qid=1699096868&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-47
88 / 244
/Wireless-Bluetooth-Headphones-Waterproof-Earphones-UK-3L/dp/B0CHRT2NYM/ref=sr_1_34?keywords=AirPods&qid=1699096868&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-34
89 / 244
/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Black/dp/B0C2KDYX4Q/ref=sr_1_33?keywords=AirPods&qid=1699096868&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-33
90 / 244
/sspa/click?ie=UTF8&spc=MTo0MDU5NDA1NzQ5NDUxNjcyOjE2OTkwOTY4Njg6c3BfYXRmX25leHQ6MzAwMDgxNzQ2MjI2OTMyOjowOjo&url=%2FFZYING-Light-Weight-Headphones-Microphone-Cancelling-R3-N1%2Fdp%2FB0CLV56GXL%2Fref%3Dsr_1_26_sspa%3Fkeywo

129 / 244
/Wireless-Earbuds-Bluetooth-Headphones-Waterproof-White/dp/B08J7B4GMC/ref=sr_1_43?keywords=AirPods&qid=1699096879&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43
130 / 244
/JBL-Tune-230NC-Ear-Headphones-Black/dp/B09HGR1DVC/ref=sr_1_30?keywords=AirPods&qid=1699096879&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-30
131 / 244
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-White/dp/B0BG579GDX/ref=sr_1_34?keywords=AirPods&qid=1699096879&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-34
132 / 244
/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Rose-Gold/dp/B0C5DKXSVD/ref=sr_1_44?keywords=AirPods&qid=1699096879&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-44
133 / 244
/Soundcore-Cancelling-AI-Enhanced-Transparency-Bluetooth-Black/dp/B09PBPDQXG/ref=sr_1_28?keywords=AirPods&qid=1699096879&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-28
134 / 244
/Bluetooth-Headphones-Waterproof-Immersive

173 / 244
/Wireless-Earbuds-Bluetooth-Headphones-Waterproof-White/dp/B08J7B4GMC/ref=sr_1_43?keywords=AirPods&qid=1699096888&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43
174 / 244
/Wireless-Headphones-Bluetooth-Cancellation-Waterproof-Pink/dp/B0BGMDVM7V/ref=sr_1_37?keywords=AirPods&qid=1699096888&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-37
175 / 244
/Bluetooth-Headphones-Waterproof-Immersive-Cancelling-white/dp/B0C7JVV8G8/ref=sr_1_29?keywords=AirPods&qid=1699096888&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29
176 / 244
/soundcore-A3919-parent-Black/dp/B07SJR6HL3/ref=sr_1_48?keywords=AirPods&qid=1699096888&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-48
177 / 244
/JLab-Wireless-Bluetooth-Earphones-Microphone-Teal/dp/B09HN4DHFL/ref=sr_1_39?keywords=AirPods&qid=1699096888&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-39
178 / 244
/Beats-Studio-Buds-Cancelling-Built/dp/B0979QQHFG/ref=sr_1_25?k

217 / 244
/JLab-Wireless-Bluetooth-Earphones-Microphone-Teal/dp/B09HN4DHFL/ref=sr_1_39?keywords=AirPods&qid=1699096894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-39
218 / 244
/Headphones-Bluetooth-Cancellation-Waterproof-Earphones-Black/dp/B0BT87ZYJG/ref=sr_1_47?keywords=AirPods&qid=1699096894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-47
219 / 244
/Wireless-Earbuds-Bluetooth-Headphones-Waterproof-White/dp/B08J7B4GMC/ref=sr_1_43?keywords=AirPods&qid=1699096894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43
220 / 244
/JBL-Tune-230NC-Ear-Headphones-Black/dp/B09HGR1DVC/ref=sr_1_30?keywords=AirPods&qid=1699096894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-30
221 / 244
/Wireless-Earbuds-Bluetooth-Headphones-Waterproof-White/dp/B08J7B4GMC/ref=sr_1_43?keywords=AirPods&qid=1699096899&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43
222 / 244
/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Black/

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,NEW Bose QuietComfort Ultra Wireless Noise Can...,299..,4.3 out of 5,310 global ratings,https://www.amazon.co.uk/Bose-QuietComfort-Can...,Value not found,Bose,Bose QuietComfort Ultra Wireless Noise Cancell...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1486,NEW Bose QuietComfort Ultra Wireless Noise Can...,299..,4.3 out of 5,310 global ratings,https://www.amazon.co.uk/Bose-QuietComfort-Can...,Value not found,Bose,Bose QuietComfort Ultra Wireless Noise Cancell...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1487,NEW Bose QuietComfort Ultra Wireless Noise Can...,299..,4.3 out of 5,310 global ratings,https://www.amazon.co.uk/Bose-QuietComfort-Can...,Value not found,Bose,Bose QuietComfort Ultra Wireless Noise Cancell...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1488,NEW Bose QuietComfort Ultra Wireless Noise Can...,299..,4.3 out of 5,310 global ratings,https://www.amazon.co.uk/Bose-QuietComfort-Can...,Value not found,Bose,Bose QuietComfort Ultra Wireless Noise Cancell...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [259]:
df.Search.value_counts()

iPhone         659
AirPods        335
iPad           261
Macbook        144
Apple Watch     81
Name: Search, dtype: int64

In [260]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=apple+watch&i=electronics&rh=n%3A560798%2Cp_72%3A419156031&dc&page=2&qid=1698925012&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    await asyncio.sleep(random.uniform(1, 5)) 
hrefs


['/Apple-Cellular-Smartwatch-Precision-Extra-Long/dp/B0CHWV2XDP/ref=sr_1_44?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-44',
 '/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYCJ1Z/ref=sr_1_26?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-26',
 '/Apple-Watch-GPS-45MM-Aluminium-Midnight/dp/B0BJBZ97HL/ref=sr_1_41?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-41',
 '/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYB4TP/ref=sr_1_35?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35',
 '/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX165HC/ref=sr_1_48?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-48',
 '/Apple-Watch-40mm-GPS-Aluminium/dp/B08QCSHB3V/ref=sr_1_47?keywords=apple+watch&qid=1699097123&refinements=p_72

In [261]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'Apple Watch'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-Cellular-Smartwatch-Precision-Extra-Long/dp/B0CHWV2XDP/ref=sr_1_44?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-44
2 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYCJ1Z/ref=sr_1_26?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-26
3 / 240
/Apple-Watch-GPS-45MM-Aluminium-Midnight/dp/B0BJBZ97HL/ref=sr_1_41?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-41
4 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYB4TP/ref=sr_1_35?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
5 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX165HC/ref=sr_1_48?keywords=apple+watch&qid=1699097123&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-48
6 / 240
/Apple-Watch-40mm-GPS-Aluminium/dp/B08QCSHB3V/ref=sr_1_47?keywords=apple+watch&qid=1

48 / 240
/Apple-Smartwatch-Starlight-Aluminum-Resistant/dp/B0CHWV6H2N/ref=sr_1_33?keywords=apple+watch&qid=1699097131&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-33
49 / 240
/Apple-Smartwatch-Midnight-Aluminum-Resistant/dp/B0CHX1J16L/ref=sr_1_37?keywords=apple+watch&qid=1699097137&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-37
50 / 240
/Apple-Watch-40mm-GPS-Aluminium/dp/B08QJQVXG5/ref=sr_1_45?keywords=apple+watch&qid=1699097137&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-45
51 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYCJ1Z/ref=sr_1_26?keywords=apple+watch&qid=1699097137&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-26
52 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWV6HBV/ref=sr_1_40?keywords=apple+watch&qid=1699097137&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40
53 / 240
/Apple-Watch-GPS-45MM-Aluminium-Midnight/dp/B0BJBZ97HL/ref=sr_1_41?keywords=apple+watch

94 / 240
/Apple-Watch-42mm-Aluminium-Renewed/dp/B07RC9B8NM/ref=sr_1_32?keywords=apple+watch&qid=1699097143&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32
95 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHWV5J6C/ref=sr_1_46?keywords=apple+watch&qid=1699097143&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-46
96 / 240
/Apple-Watch-41mm-Smart-watch/dp/B0BDJG83L4/ref=sr_1_36?keywords=apple+watch&qid=1699097143&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-36
97 / 240
/Apple-Watch-45mm-Smart-watch/dp/B0BDHQYB3Q/ref=sr_1_42?keywords=apple+watch&qid=1699097150&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-42
98 / 240
/Apple-Watch-40mm-GPS-Aluminium-Black/dp/B08QCRJ68F/ref=sr_1_43?keywords=apple+watch&qid=1699097150&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43
99 / 240
/Apple-Watch-40mm-GPS-Aluminium/dp/B08QJQVXG5/ref=sr_1_45?keywords=apple+watch&qid=1699097150&refinements=p_72%3A419156031&rni

140 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYB4TP/ref=sr_1_35?keywords=apple+watch&qid=1699097156&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
141 / 240
/Apple-Watch-41mm-Smart-watch/dp/B0BDJG83L4/ref=sr_1_36?keywords=apple+watch&qid=1699097156&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-36
142 / 240
/Fitness-Monitor-Waterproof-Counter-Smartwatch-Pink/dp/B0BR8YTL7D/ref=sr_1_30?keywords=apple+watch&qid=1699097156&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-30
143 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX165HC/ref=sr_1_48?keywords=apple+watch&qid=1699097156&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-48
144 / 240
/Apple-Watch-42mm-Aluminium-Renewed/dp/B07RC9B8NM/ref=sr_1_32?keywords=apple+watch&qid=1699097156&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32
145 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHX9GDQ1/ref=sr_1_28?keywords=apple+wat

186 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDJ7S56L/ref=sr_1_31?keywords=apple+watch&qid=1699097170&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-31
187 / 240
/Apple-Smartwatch-Starlight-Aluminum-Resistant/dp/B0CHWV6H2N/ref=sr_1_34?keywords=apple+watch&qid=1699097170&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-34
188 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYB4TP/ref=sr_1_35?keywords=apple+watch&qid=1699097170&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
189 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX165HC/ref=sr_1_48?keywords=apple+watch&qid=1699097170&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-48
190 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHX9GDQ1/ref=sr_1_28?keywords=apple+watch&qid=1699097170&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-28
191 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDJ3T7MW/ref=sr_1_38?keywords=apple+wa

232 / 240
/Apple-Smartwatch-Starlight-Aluminum-Resistant/dp/B0CHWV6H2N/ref=sr_1_33?keywords=apple+watch&qid=1699097183&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-33
233 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHWV5J6C/ref=sr_1_46?keywords=apple+watch&qid=1699097183&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-46
234 / 240
/Apple-Watch-41mm-Smart-watch/dp/B0BDJG83L4/ref=sr_1_36?keywords=apple+watch&qid=1699097183&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-36
235 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWV6HBV/ref=sr_1_40?keywords=apple+watch&qid=1699097183&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40
236 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWYCJ1Z/ref=sr_1_26?keywords=apple+watch&qid=1699097183&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-26
237 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHWYF4G1/ref=sr_1_38?keywords=ap

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,"Apple Watch Series 8 (GPS, 45MM) - Midnight Al...",288..,4.4 out of 5,39 global ratings,https://www.amazon.co.uk/Apple-Watch-GPS-45MM-...,Value not found,Apple,Series 8,NaN,NaN,...,Unisex Adults,NaN,Analog,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1510,"Apple Watch Series 8 (GPS, 45MM) - Midnight Al...",288..,4.4 out of 5,39 global ratings,https://www.amazon.co.uk/Apple-Watch-GPS-45MM-...,Value not found,Apple,Series 8,NaN,NaN,...,Unisex Adults,NaN,Analog,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1511,"Apple Watch Series 8 (GPS, 45MM) - Midnight Al...",288..,4.4 out of 5,39 global ratings,https://www.amazon.co.uk/Apple-Watch-GPS-45MM-...,Value not found,Apple,Series 8,NaN,NaN,...,Unisex Adults,NaN,Analog,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1512,"Apple Watch Series 8 (GPS, 45MM) - Midnight Al...",288..,4.4 out of 5,39 global ratings,https://www.amazon.co.uk/Apple-Watch-GPS-45MM-...,Value not found,Apple,Series 8,NaN,NaN,...,Unisex Adults,NaN,Analog,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
df.Search.value_counts()

iPhone         659
AirPods        335
iPad           261
Macbook        144
Apple Watch    101
Name: Search, dtype: int64

In [ ]:
Mac
iPhone
iPad
AirPods
Watch


In [175]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

class AmazonScraper:
    def __init__(
        self,
        headless=True,
        load_images=False,
        chromedriver_path="./chromedriver",
        window_size=(700,900),
        ):
        options = webdriver.ChromeOptions()
        
        if headless:
            options.add_argument("--headless")
            options.add_argument("--disable-gpu")
        
        if not load_images:
            prefs = {"profile.managed_default_content_settings.images": 2}
            options.add_experimental_option("prefs", prefs)
        
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(
            executable_path=chromedriver_path, options=options
        )
        self.driver.set_window_size(*window_size)
            

In [180]:
scraper = AmazonScraper(headless=False)

In [188]:
scraper.driver.get('https://www.amazon.co.uk/Apple-iPhone-11-64GB-White/dp/B08L6XKFRY/ref=cm_cr_arp_d_product_top?ie=UTF8'
                  )

In [192]:
scraper.driver.find_element_by_class_name('a-size-base review-text').text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".a-size-base review-text"}
  (Session info: chrome=119.0.6045.105)
